In [ ]:
!pip install toad

In [ ]:
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
#         elif t == np.object:
#             if cols[i] == 'date':
#                 df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
#             else:
#                 df[cols[i]] = df[cols[i]].astype('category')
    return df  

In [ ]:
import pandas as pd
import gc
import numpy as np
import catboost as cbt
from tqdm import tqdm
import lightgbm as lgb
from datetime import datetime, timedelta
from scipy.special import boxcox1p
from scipy.stats import skew, norm
from sklearn.model_selection import train_test_split
from scipy.stats import boxcox_normmax
import category_encoders as ce
from collections import Counter
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import  roc_auc_score,accuracy_score
import warnings
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler,StandardScaler
#import toad
warnings.filterwarnings('ignore')

In [ ]:
# 读取数据
train_op = pd.read_csv('/kaggle/input/yizhifu-7-20/train_op.csv')
df_train_label = pd.read_csv('/kaggle/input/yizhifu-7-20/train_label.csv')
df_train_base = pd.read_csv('/kaggle/input/yizhifu-7-20/train_base.csv')
df_train_trans = pd.read_csv('/kaggle/input/yizhifu-7-20/train_trans.csv')


testA

In [ ]:
test_op = pd.read_csv('../input/yizhifu-7-20/test_a_op.csv')
df_test_base = pd.read_csv('../input/yizhifu-7-20/test_a_base.csv')
df_test_trans = pd.read_csv('../input/yizhifu-7-20/test_a_trans.csv')
submit = pd.read_csv('../input/yizhifu-7-20/submit_example.csv')

testB

In [ ]:
test_op = pd.read_csv('../input/fusaishuju/testb_op.csv')
df_test_base = pd.read_csv('../input/fusaishuju/testb_base.csv')
df_test_trans = pd.read_csv('../input/fusaishuju/testb_trans.csv')
submit = pd.read_csv('../input/fusaishuju/submit_example.csv')

In [ ]:
df_trans = df_train_trans.append(df_test_trans)
df_trans = df_trans.reset_index(drop=True)
df_op = train_op.append(test_op)
df_op = df_op.reset_index(drop=True)

In [ ]:
def parse_time(tm):
    days, _, time = tm.split(' ')
    time = time.split('.')[0]
    time = '2020-1-1 ' + time
    time = datetime.strptime(time, '%Y-%m-%d %H:%M:%S')
    time = (time + timedelta(days=int(days)))
    return time

df_trans['date'] = df_trans['tm_diff'].apply(parse_time)
df_op['date'] = df_op['tm_diff'].apply(parse_time)

In [ ]:
def date_deal(train_te):
    # 日
    train_te['day']=train_te['date'].apply(lambda x: x.day)
    # 小时
    train_te['hour']=train_te['date'].apply(lambda x: x.hour)
    # 分钟
    train_te['minute']=train_te['date'].apply(lambda x: x.minute)
    # 秒数
    train_te['Seconds']=train_te['date'].apply(lambda x: x.second)
    # 一天中的第几分钟
    train_te['Minutes_of_the_day']=train_te['date'].apply(lambda x: x.minute + x.hour*60)
    # 星期几
    train_te['Day_of_the_week']=train_te['date'].apply(lambda x: x.dayofweek )
    return train_te
df_trans = date_deal(df_trans)
df_op = date_deal(df_op)

In [ ]:
df_op.sort_values(['user', 'date'], inplace=True)
df_op = df_op.reset_index(drop=True)
df_trans.sort_values(['user', 'date'], inplace=True)
df_trans = df_trans.reset_index(drop=True)

In [ ]:
df_train = df_train_base.merge(df_train_label, how='left')
df_test = df_test_base
df_feature = df_train.append(df_test)

In [ ]:
def count_ft(df_feature1 = df_feature):
    df_temp = df_trans.groupby(['user'])['amount'].agg(amount_mean='mean',amount_std='std',amount_sum='sum',amount_max='max',amount_min='min',amount_med='median',amount_cnt='count').reset_index()
    df_feature1 = df_feature1.merge(df_temp, how='left')
    df_temp = df_trans.groupby(['user'])['day'].agg(day_mean='mean',day_std='std',day_sum='sum',day_max='max',day_min='min').reset_index()
    df_feature1 = df_feature1.merge(df_temp, how='left')
    df_temp = df_trans.groupby(['user'])['hour'].agg(hour_mean='mean',hour_std='std',hour_sum='sum',hour_max='max',hour_min='min').reset_index()
    df_feature1 = df_feature1.merge(df_temp, how='left')
    df_temp = df_trans.groupby(['user'])['minute'].agg(minute_mean='mean',minute_std='std',minute_sum='sum',minute_max='max',minute_min='min').reset_index()
    df_feature1 = df_feature1.merge(df_temp, how='left')
    df_temp = df_trans.groupby(['user'])['Seconds'].agg(Seconds_mean='mean',Seconds_std='std',Seconds_sum='sum',Seconds_max='max',Seconds_min='min').reset_index()
    df_feature1 = df_feature1.merge(df_temp, how='left')
    df_temp = df_trans.groupby(['user'])['Minutes_of_the_day'].agg(Minutes_of_the_day_mean='mean',Minutes_of_the_day_std='std',Minutes_of_the_day_sum='sum',Minutes_of_the_day_max='max',Minutes_of_the_day_min='min').reset_index()
    df_feature1 = df_feature1.merge(df_temp, how='left')
    df_temp = df_trans.groupby(['user'])['Day_of_the_week'].agg(Day_of_the_week_mean='mean',Day_of_the_week_std='std',Day_of_the_week_sum='sum',Day_of_the_week_max='max',Day_of_the_week_min='min').reset_index()
    df_feature1 = df_feature1.merge(df_temp, how='left')
#     df_temp = df_trans.groupby(['user'])['type_proportion'].agg(Day_of_the_week_mean='mean',Day_of_the_week_std='std',Day_of_the_week_sum='sum',Day_of_the_week_max='max',Day_of_the_week_min='min').reset_index()
#     df_feature1 = df_feature1.merge(df_temp, how='left')
    
    return df_feature1
df_feature = count_ft()

In [ ]:
def count1_ft(df_feature1 = df_feature):
    df_temp = df_op.groupby(['user'])['day'].agg(day_mean1='mean',day_std1='std',day_sum1='sum',day_max1='max',day_min1='min').reset_index()
    df_feature1 = df_feature1.merge(df_temp, how='left')
    df_temp = df_op.groupby(['user'])['hour'].agg(hour_mean1='mean',hour_std1='std',hour_sum1='sum',hour_max1='max',hour_min1='min').reset_index()
    df_feature1 = df_feature1.merge(df_temp, how='left')
    df_temp = df_op.groupby(['user'])['minute'].agg(minute_mean1='mean',minute_std1='std',minute_sum1='sum',minute_max1='max',minute_min1='min').reset_index()
    df_feature1 = df_feature1.merge(df_temp, how='left')
    df_temp = df_op.groupby(['user'])['Seconds'].agg(Seconds_mean1='mean',Seconds_std1='std',Seconds_sum1='sum',Seconds_max1='max',Seconds_min1='min').reset_index()
    df_feature1 = df_feature1.merge(df_temp, how='left')
    df_temp = df_op.groupby(['user'])['Minutes_of_the_day'].agg(Minutes_of_the_day_mean1='mean',Minutes_of_the_day_std1='std',Minutes_of_the_day_sum1='sum',Minutes_of_the_day_max1='max',Minutes_of_the_day_min1='min').reset_index()
    df_feature1 = df_feature1.merge(df_temp, how='left')
    df_temp = df_op.groupby(['user'])['Day_of_the_week'].agg(Day_of_the_week_mean1='mean',Day_of_the_week_std1='std',Day_of_the_week_sum1='sum',Day_of_the_week_max1='max',Day_of_the_week_min1='min').reset_index()
    df_feature1 = df_feature1.merge(df_temp, how='left')
    
    return df_feature1
df_feature = count1_ft()

In [ ]:
df_trans['days_diff'] = df_trans['tm_diff'].apply(lambda x: int(x.split(' ')[0]))

In [ ]:
def w2v_feat(df, group_id, feat, length):
    print('start word2vec ...')
    df[feat] = df[feat].astype('str')
    data_frame = df.groupby(group_id)[feat].agg(list).reset_index()
    model = Word2Vec(data_frame[feat].values, size=length, window=5, min_count=1, sg=1,workers=1, hs=1, iter=10, seed=1)
    data_frame[feat] = data_frame[feat].apply(lambda x: pd.DataFrame([model[c] for c in x]))
    for m in range(length): 
        data_frame['w2v_{}_mean'.format(m)] = data_frame[feat].apply(lambda x: x[m].mean())
    del data_frame[feat]
    return data_frame
df_feature = df_feature.merge(w2v_feat(df_trans, 'user', 'amount', 32), on=['user'], how='left')
df_feature = df_feature.merge(w2v_feat(df_op, 'user', 'op_device', 32), on=['user'], how='left')

In [ ]:
df_feature = df_feature.merge(w2v_feat(df_op, 'user', 'ip', 32), on=['user'], how='left')

In [ ]:
# def lda_feature(prefix,df, groupby, target,n_topic):
#     df_bag = pd.DataFrame(df[[groupby, target]])
#     df_bag[target] = df_bag[target].astype(str)
#     df_bag[target].fillna('NAN', inplace=True)    
#     df_bag = df_bag.groupby(groupby, as_index=False)[target].agg({'list':(lambda x: list(x))})
#     df_bag['sentence'] = df_bag['list'].apply(lambda x: list(map(str,x)))
#     docs = df_bag['sentence'].tolist() 
#     dictionary = corpora.Dictionary(docs)
#     corpus = [dictionary.doc2bow(text) for text in docs]
#     lda = LdaMulticore(corpus, id2word=dictionary, num_topics=n_topic)
#     docres = [dict(lda[doc_bow]) for doc_bow in corpus]
#     df_lda = pd.DataFrame(docres,dtype=np.float16).fillna(0.001)
#     df_lda.columns = ['lda_%s_%s_%d'%(prefix,target,x) for x in range(n_topic)]
#     df_lda[groupby] = df_bag[groupby]
#     print ('df_lda:' + str(df_lda.shape))
#     return df_lda
# df_feature = df_feature.merge(lda_feature('lda',df_trans, 'user', 'amount', 10), on=['user'], how='left')

In [ ]:
#df_feature['amount_per_days'] = df_feature['amount_sum'] / df_feature['user_trans_days_diff_nuniq']
#df_feature['amount_per_cnt'] = df_feature['amount_sum'] / df_feature['amount_cnt']

In [ ]:
# group_df = df_trans[df_trans['type1']=='45a1168437c708ff'].groupby(['user'])['days_diff'].agg(user_type1_45a1168437c708ff_min_day= 'min').reset_index()
# df_feature = df_feature.merge(group_df, on=['user'], how='left')

In [ ]:
# def gen_user_null_features(df, value, prefix):
#     df['is_null'] = 0
#     df.loc[df[value].isnull(), 'is_null'] = 1
#     group_df = df.groupby(['user'])['is_null'].agg(user_trans_ip_null_cnt= 'sum',
#                                                    user_trans_ip_null_ratio= 'mean').reset_index()
#     return group_df
# df_feature = df_feature.merge(gen_user_null_features(df=df_trans, value='ip', prefix='trans'), on=['user'], how='left')

In [ ]:
def gen_user_tfidf_features(df, value):
    df[value] = df[value].astype(str)
    df[value].fillna('-1', inplace=True)
    group_df = df.groupby(['user']).apply(lambda x: x[value].tolist()).reset_index()
    group_df.columns = ['user', 'list']
    group_df['list'] = group_df['list'].apply(lambda x: ','.join(x))
    enc_vec = TfidfVectorizer()
    tfidf_vec = enc_vec.fit_transform(group_df['list'])
    svd_enc = TruncatedSVD(n_components=10, n_iter=20, random_state=2020)
    vec_svd = svd_enc.fit_transform(tfidf_vec)
    vec_svd = pd.DataFrame(vec_svd)
    vec_svd.columns = ['svd_tfidf_{}_{}'.format(value, i) for i in range(10)]
    group_df = pd.concat([group_df, vec_svd], axis=1)
    del group_df['list']
    return group_df

def gen_user_countvec_features(df, value):
    df[value] = df[value].astype(str)
    df[value].fillna('-1', inplace=True)
    group_df = df.groupby(['user']).apply(lambda x: x[value].tolist()).reset_index()
    group_df.columns = ['user', 'list']
    group_df['list'] = group_df['list'].apply(lambda x: ','.join(x))
    enc_vec = CountVectorizer()
    tfidf_vec = enc_vec.fit_transform(group_df['list'])
    svd_enc = TruncatedSVD(n_components=10, n_iter=20, random_state=2020)
    vec_svd = svd_enc.fit_transform(tfidf_vec)
    vec_svd = pd.DataFrame(vec_svd)
    vec_svd.columns = ['svd_countvec_{}_{}'.format(value, i) for i in range(10)]
    group_df = pd.concat([group_df, vec_svd], axis=1)
    del group_df['list']
    return group_df

In [ ]:
df_feature = df_feature.merge(gen_user_tfidf_features(df=df_op, value='op_mode'), on=['user'], how='left')
df_feature = df_feature.merge(gen_user_tfidf_features(df=df_op, value='op_type'), on=['user'], how='left')
df_feature = df_feature.merge(gen_user_countvec_features(df=df_op, value='op_mode'), on=['user'], how='left')
df_feature = df_feature.merge(gen_user_countvec_features(df=df_op, value='op_type'), on=['user'], how='left')

In [ ]:
df_feature = df_feature.merge(gen_user_tfidf_features(df=df_op, value='ip'), on=['user'], how='left')
df_feature = df_feature.merge(gen_user_countvec_features(df=df_op, value='ip'), on=['user'], how='left')

In [ ]:
def gen_user_group_amount_features(df, value):
    group_df = df.pivot_table(index='user',
                              columns=value,
                              values='amount',
                              dropna=False,
                              aggfunc=['count', 'sum']).fillna(0)
    group_df.columns = ['user_{}_{}_amount_{}'.format(value, f[1], f[0]) for f in group_df.columns]
    group_df.reset_index(inplace=True)
    return group_df 
df_trans['amount'] = df_trans['amount'].astype('int')
df_feature = df_feature.merge(gen_user_group_amount_features(df=df_trans, value='platform'), on=['user'], how='left')
df_feature = df_feature.merge(gen_user_group_amount_features(df=df_trans, value='type1'), on=['user'], how='left')
df_feature = df_feature.merge(gen_user_group_amount_features(df=df_trans, value='type2'), on=['user'], how='left')
df_feature = df_feature.merge(gen_user_group_amount_features(df=df_trans, value='tunnel_in'), on=['user'], how='left')
df_feature = df_feature.merge(gen_user_group_amount_features(df=df_trans, value='tunnel_out'), on=['user'], how='left')


In [ ]:
# df_feature = df_feature.merge(gen_user_group_amount_features(df=df_trans, value='type1'), on=['user'], how='left')
# df_feature = df_feature.merge(gen_user_group_amount_features(df=df_trans, value='type2'), on=['user'], how='left')

In [ ]:
def gen_user_window27_amount_features(df, window):
    group_df = df[df['days_diff']>window].groupby('user')['amount'].agg(
        user_amount_mean_27 = 'mean',
        user_amount_std_27 = 'std',
        user_amount_max_27 = 'max',
        user_amount_min_27 = 'min',
        user_amount_sum_27 = 'sum',
        user_amount_med_27 = 'median',
        user_amount_cnt_27 = 'count',
        ).reset_index()
    return group_df
def gen_user_window23_amount_features(df, window):
    group_df = df[df['days_diff']>window].groupby('user')['amount'].agg(
        user_amount_mean_23 = 'mean',
        user_amount_std_23 = 'std',
        user_amount_max_23 = 'max',
        user_amount_min_23 = 'min',
        user_amount_sum_23 = 'sum',
        user_amount_med_23 = 'median',
        user_amount_cnt_23 = 'count',
        ).reset_index()
    return group_df
def gen_user_window15_amount_features(df, window):
    group_df = df[df['days_diff']>window].groupby('user')['amount'].agg(
        user_amount_mean_15 = 'mean',
        user_amount_std_15 = 'std',
        user_amount_max_15 = 'max',
        user_amount_min_15 = 'min',
        user_amount_sum_15 = 'sum',
        user_amount_med_15 = 'median',
        user_amount_cnt_15 = 'count',
        ).reset_index()
    return group_df

df_trans['amount'] = df_trans['amount'].astype('int')
df_trans['days_diff'] = df_trans['days_diff'].astype('int')
df_feature = df_feature.merge(gen_user_window27_amount_features(df=df_trans, window=27), on=['user'], how='left')
df_feature = df_feature.merge(gen_user_window23_amount_features(df=df_trans, window=23), on=['user'], how='left')
df_feature = df_feature.merge(gen_user_window15_amount_features(df=df_trans, window=15), on=['user'], how='left')
df_feature = df_feature.merge(gen_user_window15_amount_features(df=df_trans, window=9), on=['user'], how='left')
df_feature = df_feature.merge(gen_user_window15_amount_features(df=df_trans, window=2), on=['user'], how='left')
#df_feature = df_feature.merge(gen_user_window185_amount_features(df=df_trans, window1=5,window2=28), on=['user'], how='left')

In [ ]:
for f in [
        'balance', 'balance_avg', 'balance1', 'balance1_avg', 'balance2',
        'balance2_avg', 'product1_amount', 'product2_amount',
        'product3_amount', 'product4_amount', 'product5_amount', 'product6_amount'
]:
    df_feature[f] = df_feature[f].apply(lambda x: int(x.split(' ')[1]) if type(x) != float else np.NaN)

In [ ]:
df_feature['product7_fail_ratio'] = df_feature['product7_fail_cnt'] / df_feature['product7_cnt']
df_feature['card_cnt'] = df_feature['card_a_cnt'] + df_feature['card_b_cnt'] + df_feature['card_c_cnt'] + df_feature['card_d_cnt']
df_feature['acc_card_ratio'] = df_feature['acc_count'] / df_feature['card_cnt']
df_feature['login_cnt'] = df_feature['login_cnt_period1'] + df_feature['login_cnt_period2']

In [ ]:
df_feature['city_level'] = df_feature['city'].map(str) + '_' + df_feature['level'].map(str)
df_feature['city_balance_avg'] = df_feature['city'].map(str) + '_' + df_feature['balance_avg'].map(str)

In [ ]:
def stat(df, df_merge, group_by, agg):
    group = df.groupby(group_by).agg(agg)
    columns = []
    for on, methods in agg.items():
        for method in methods:
            columns.append('{}_{}_{}'.format('_'.join(group_by), on, method))
    group.columns = columns
    group.reset_index(inplace=True)
    df_merge = df_merge.merge(group, on=group_by, how='left')
    del (group)
    gc.collect()
    return df_merge

def statis_feat(df_know, df_unknow):   
    df_unknow = stat(df_know, df_unknow, ['province'], {'label': ['mean']})
    df_unknow = stat(df_know, df_unknow, ['city'], {'label': ['mean']})
    df_unknow = stat(df_know, df_unknow, ['city_level'], {'label': ['mean']})
    df_unknow = stat(df_know, df_unknow, ['city_balance_avg'], {'label': ['mean']})
#     df_unknow = stat(df_know, df_unknow, ['cross_province_level'], {'label': ['mean']})
#     df_unknow = stat(df_know, df_unknow, ['cross_province_verified'], {'label': ['mean']})
#     df_unknow = stat(df_know, df_unknow, ['cross_province_regist_type'], {'label': ['mean']})
#     df_unknow = stat(df_know, df_unknow, ['cross_province_service3'], {'label': ['mean']})
    return df_unknow

df_train = df_feature[~df_feature['label'].isnull()]
df_train = df_train.reset_index(drop=True)
df_test = df_feature[df_feature['label'].isnull()]

df_stas_feat = None
kf = StratifiedKFold(n_splits=5, random_state=2020, shuffle=True)
for train_index, val_index in kf.split(df_train, df_train['label']):
    df_fold_train = df_train.iloc[train_index]
    df_fold_val = df_train.iloc[val_index]
    df_fold_val = statis_feat(df_fold_train, df_fold_val)
    df_stas_feat = pd.concat([df_stas_feat, df_fold_val], axis=0)
    del (df_fold_train)
    del (df_fold_val)
    gc.collect()

df_test = statis_feat(df_train, df_test)
df_feature = pd.concat([df_stas_feat, df_test], axis=0)
df_feature = df_feature.reset_index(drop=True)

del (df_stas_feat)
del (df_train)
del (df_test)
del df_feature['service3_level']
del df_feature['city_level']
del df_feature['city_balance_avg']
gc.collect()

In [ ]:
# for c in tqdm(['sex','provider','level','verified','regist_type','service3']):
#     for c1 in ['sex','provider','level','verified','regist_type','service3']:
#         if c1==c:
#             continue
#         if 'cross_{}_{}'.format(c,c1) not in df_feature.columns and 'cross_{}_{}'.format(c1,c) not in df_feature.columns:
#             df_feature['cross_{}_{}'.format(c,c1)]=df_feature[c].astype(str)+df_feature[c1].astype(str)

In [ ]:
for i in tqdm([i for i in df_feature.select_dtypes('object').columns if i != 'user']):
    df_feature[f'{i}_count'] = df_feature.groupby([i])['user'].transform('count')
    
# cross_cols = [i for i in df_feature.select_dtypes('object').columns if i != 'user']
# for f in tqdm(cross_cols):
#     for col in cross_cols:
#         if col == f:
#             continue
#         if  'cross_{}_{}_nunique'.format(f, col) not in df_feature.columns.values:
#             df_feature['cross_{}_{}_nunique'.format(f, col)] = df_feature.groupby(f)[col].transform('nunique')
#         if 'cross_{}_{}_count'.format(f, col) not in df_feature.columns.values and 'cross_{}_{}_count'.format(col, f) not in df_feature.columns.values:
#             df_feature['cross_{}_{}_count_ratio'.format(f, col)] = df_feature.groupby([f,col])['user'].transform('count')/df_feature[f + '_count']
#         if 'cross_{}_{}_count_ratio'.format(col, f) not in df_feature.columns.values:
#             df_feature['cross_{}_{}_count_ratio'.format(col, f)] = df_feature['cross_{}_{}_count'.format(f, col)] / df_feature[f + '_count'] # 比例偏好
#         if 'cross_{}_{}_count_ratio'.format(f, col) not in df_feature.columns.values:
#             df_feature['cross_{}_{}_count_ratio'.format(f, col)] = df_feature['cross_{}_{}_count'.format(f, col)] / df_feature[col + '_count'] # 比例偏好
#         df_feature['cross_{}_{}_nunique_ratio_{}_count'.format(f, col, f)] = df_feature['cross_{}_{}_nunique'.format(f, col)] / df_feature[f + '_count']


In [ ]:
for f in tqdm(df_feature.select_dtypes('object').columns):
    if f not in ['user']:
        lbl = LabelEncoder()
        df_feature[f] = lbl.fit_transform(df_feature[f].astype(str))

In [ ]:
df_feature['rate_agreement'] = (df_feature['agreement1']+df_feature['agreement2']+df_feature['agreement3']+df_feature['agreement4'])/df_feature['agreement_total']
df_feature['cart'] = df_feature['sex']+df_feature['age']+df_feature['balance']+df_feature['regist_type']
df_feature['cart1'] = df_feature['sex']+df_feature['age']
df_feature['cart2'] = df_feature['sex']+df_feature['balance']+df_feature['regist_type']
df_feature['cart2'] = df_feature['balance']+df_feature['regist_type']
df_feature['card_a_rate'] = df_feature['card_a_cnt'] / df_feature['card_cnt'] 
df_feature['card_b_rate'] = df_feature['card_b_cnt'] / df_feature['card_cnt'] 
df_feature['card_c_rate'] = df_feature['card_c_cnt'] / df_feature['card_cnt'] 
df_feature['card_d_rate'] = df_feature['card_d_cnt'] / df_feature['card_cnt'] 
df_feature['op_cnt'] = df_feature['op1_cnt'] + df_feature['op2_cnt'] 
df_feature['op1_cnt_rate'] = df_feature['op1_cnt'] / df_feature['op_cnt'] 
df_feature['op2_cnt_rate'] = df_feature['op2_cnt'] / df_feature['op_cnt'] 

In [ ]:
df_train = df_feature[~df_feature['label'].isnull()].copy()
df_test = df_feature[df_feature['label'].isnull()].copy()

In [ ]:
df_train = df_feature[:df_train_base.shape[0]]
df_test = df_feature[df_train_base.shape[0]:]

In [ ]:
Counter(df_train['label'])

In [ ]:
df_feature.to_csv('df_feature_test1.csv',index=False)

In [ ]:
# df_feature['province_nunique_regist_type'] = df_feature.groupby('province')['regist_type'].transform('nunique')
# df_feature['province_nunique_provider'] = df_feature.groupby('province')['provider'].transform('nunique')
# df_feature['login1_cnt_rate'] = df_feature['login_cnt_period1'] / df_feature['login_cnt']
# df_feature['login2_cnt_rate'] = df_feature['login_cnt_period2'] / df_feature['login_cnt']
# df_feature['balance1_rate'] = df_feature['balance1']/df_feature['balance1_avg']
# df_feature['balance2_rate'] = df_feature['balance2']/df_feature['balance2_avg']

### 贪婪算法lr筛选特征

In [ ]:
# """
# Greedy Feature Selection using Logistic Regression as base model
# to optimize Area Under the ROC Curve

# """
# import numpy as np

# import sklearn.linear_model as lm
# from sklearn import metrics, preprocessing


# class greedyFeatureSelection(object):

#     def __init__(self, data, labels, scale=1, verbose=0):
#         if scale == 1:
#             self._data = preprocessing.scale(np.array(data))
#         else:
#             self._data = np.array(data)
#         self._labels = labels
#         self._verbose = verbose

#     def evaluateScore(self, X, y):
#         x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2020, stratify=y)
#         model = lm.LogisticRegression()
#         model.fit(x_train, y_train)
#         predictions = model.predict_proba(x_test)[:, 1]
#         auc = metrics.roc_auc_score(y_test, predictions)
#         return auc

#     def selectionLoop(self, X, y):
#         score_history = []
#         good_features = set([])
#         num_features = X.shape[1]
#         while len(score_history) < 2 or score_history[-1][0] > score_history[-2][0]:
#             scores = []
#             for feature in range(num_features):
#                 if feature not in good_features:
#                     selected_features = list(good_features) + [feature]

#                     Xts = np.column_stack(X[:, j] for j in selected_features)

#                     score = self.evaluateScore(Xts, y)
#                     scores.append((score, feature))

#                     if self._verbose:
#                         print ("Current AUC : ", np.mean(score))

#             good_features.add(sorted(scores)[-1][1])
#             score_history.append(sorted(scores)[-1])
#             if self._verbose:
#                 print ("Current Features : ", sorted(list(good_features)))

#         # Remove last added feature
#         good_features.remove(score_history[-1][1])
#         good_features = sorted(list(good_features))
#         if self._verbose:
#             print ("Selected Features : ", good_features)

#         return good_features

#     def transform(self, X):
#         X = self._data
#         y = self._labels
#         good_features = self.selectionLoop(X, y)
#         return X[:, good_features]

# col = [i for i in df_train.columns if i not in ['user','label']]
# X_train = df_train[col]
# #X_train['oof'] = oof
# y = df_train['label']
# X_test = df_test[col]
# X_train.fillna(X_train.mean(),inplace=True)
# X_test.fillna(X_test.mean(),inplace=True)
# x = greedyFeatureSelection(data=X_train, labels=y,verbose=1)
# X = x.transform(X_train)

In [ ]:
df_feature = pd.read_csv('../input/df-feature-yizhifu/df_feature_test2.csv')

In [ ]:
#初赛伪标签
df_feature = pd.read_csv('../input/df-feature-yizhifu/df_feature_test1.csv')
df_test_1 = df_feature[df_feature['label'].isnull()].copy()
submits = pd.read_csv('../input/1231412/file00525306.csv')
df_test1 = df_test_1.merge(submits, on=['user'], how='left')
df1 = df_test1[df_test1['prob']>0.840]
df1['label'] = 1
del df1['prob']
df_train1 = pd.concat([df1,df_train],axis = 0)

In [ ]:
#复赛伪标签
submits = pd.read_csv('../input/1231412/submit1 (28).csv')
df_test1 = df_test.merge(submits, on=['user'], how='left')
df1 = df_test1[df_test1['prob']>0.840]
df1['label'] = 1
del df1['prob']
df_train2 = pd.concat([df1,df_train],axis = 0)

In [ ]:
# ####catboost模型融合了五个种子会比较慢
# cat = [i for i in df_train.columns if df_train[i].dtype=='object' and i not in ['set','user','label']]
# col = [i for i in df_train.columns if i not in ['user','set','label']+cat]
# for f in tqdm(col):
#     df_train[f] = df_train[f].fillna(-1).astype('int32')
#     df_test[f] = df_test[f].fillna(-1).astype('int32')
# for f in tqdm(cat):
#     df_train[f] = df_train[f].fillna(-1)
#     df_test[f] = df_test[f].fillna(-1)
# X_train = df_train[col+cat]
# y = df_train['label']
# X_test = df_test[col+cat]


# print(X_train.shape,X_test.shape)
# oof = np.zeros(X_train.shape[0])
# prediction = np.zeros(X_test.shape[0])
# seeds = [1024]#19970412, 2019 * 2 + 1024, 4096, 2048, 
# num_model_seed = 1
# for model_seed in range(num_model_seed):
#     oof_cat = np.zeros(X_train.shape[0])
#     prediction_cat=np.zeros(X_test.shape[0])
#     skf = StratifiedKFold(n_splits=5, random_state=seeds[model_seed], shuffle=True)
#     for index, (train_index, test_index) in enumerate(skf.split(X_train, y)):
#         print(index)
#         train_x, test_x, train_y, test_y = X_train.iloc[train_index], X_train.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
#         cbt_model = cbt.CatBoostClassifier(iterations=10000,learning_rate=0.5,verbose=100,max_depth=7,
#                                        early_stopping_rounds=100,task_type='GPU',eval_metric='AUC',
                                      
#                                        )#
#         cbt_model.fit(train_x, train_y  ,eval_set=(test_x,test_y),cat_features=cat+col)
#         oof_cat[test_index] += cbt_model.predict_proba(test_x)[:,1]
#         prediction_cat += cbt_model.predict_proba(X_test)[:,1]/5
#     print('AUC',roc_auc_score(y, oof_cat))
#     oof += oof_cat / num_model_seed
#     prediction += prediction_cat / num_model_seed
# print('AUC',roc_auc_score(y, oof))

In [ ]:
# # 3层神经网络模型融合了五个种子
# col = [i for i in df_train.columns if i not in ['user','label']]
# X_train = df_train1[col]
# #X_train['oof'] = oof
# y = df_train1['label']
# X_test = df_test[col]

# def formatTrans(df,columns):
#     for i in col:
#         if df[i].dtypes == 'float64':
#             df[i] = df[i].astype(np.float32)
#         if df[i].dtypes == 'int64':
#             df[i] = df[i].astype(np.int32)
# formatTrans(X_train,col)
# formatTrans(X_test,col)

# scalar=StandardScaler() 
# for i in col:
#     X_train[i]=scalar.fit_transform(X_train[i].values.reshape(-1, 1))
#     X_test[i]=scalar.transform(X_test[i].values.reshape(-1, 1))
# # 数据归一化
# # scalar=MinMaxScaler()
# # X_train = scalar.fit_transform(X_train)
# # X_test = scalar.transform(X_test)
# X_train.fillna(-1,inplace=True)
# X_test.fillna(-1,inplace=True)
# # 3层神经网络
# from sklearn.neural_network import MLPClassifier
# seeds = [19970412, 2020 * 2 + 1024, 4096, 2048,1024]# 
# num_model_seed = 5
# for i,model_seed in enumerate(range(num_model_seed)):
#     oof_cat = np.zeros(X_train.shape[0])
#     prediction_cat=np.zeros(X_test.shape[0])
#     skf = StratifiedKFold(n_splits=5, random_state=seeds[model_seed], shuffle=True)
#     print(f'第{i}个种子：{model_seed}')
#     for index, (train_index, test_index) in enumerate(skf.split(X_train, y)):
#         train_x, test_x, train_y, test_y = X_train.iloc[train_index], X_train.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
#         dnn = MLPClassifier(hidden_layer_sizes =(2**10,2**9,2**5),verbose=True,max_iter=1000,random_state=2020)                    # criterion='mae'
#         dnn.fit(train_x,train_y)
#         oof_cat[test_index] += dnn.predict_proba(test_x)[:,1]
#         prediction_cat += dnn.predict_proba(X_test)[:, 1]/5
#     print('AUC',roc_auc_score(y, oof_cat)) 
#     prediction += prediction_cat / num_model_seed
# print('AUC',roc_auc_score(y, oof))


In [ ]:
# # svm模型融合了五个种子

# from sklearn import svm
# seeds = [19970412, 2020 * 2 + 1024, 4096, 2048,1024]# 
# num_model_seed = 5
# for i,model_seed in enumerate(range(num_model_seed)):
#     oof_cat = np.zeros(X_train.shape[0])
#     prediction_cat=np.zeros(X_test.shape[0])
#     skf = StratifiedKFold(n_splits=5, random_state=seeds[model_seed], shuffle=True)
#     print(f'第{i}个种子：{model_seed}')
#     for index, (train_index, test_index) in enumerate(skf.split(X_train, y)):
#         print(index)
#         train_x, test_x, train_y, test_y = X_train.iloc[train_index], X_train.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
#         dnn = svm.SVC(probability = True,random_state=2020)                    # criterion='mae'
#         dnn.fit(train_x,train_y)
#         oof_cat[test_index] += dnn.predict_proba(test_x)[:,1]
#         prediction_cat += dnn.predict_proba(X_test)[:, 1]/5
#     print('AUC',roc_auc_score(y, oof_cat)) 
#     prediction += prediction_cat / num_model_seed
# print('AUC',roc_auc_score(y, oof))


In [ ]:
# # 3层神经网络
# import numpy as np
# import pandas as pd
# from keras.models import Sequential, Model
# from keras.layers import Input, Dense, Embedding, Concatenate, Flatten, BatchNormalization, Dropout, concatenate
# from keras.callbacks import ModelCheckpoint
# import tensorflow as tf
# import random


# col = [i for i in df_train.columns if i not in ['user','label']]
# X_train = df_train[col]
# #X_train['oof'] = oof
# y = df_train['label']
# X_test = df_test[col]

# def formatTrans(df,columns):
#     for i in col:
#         if df[i].dtypes == 'float64':
#             df[i] = df[i].astype(np.float32)
#         if df[i].dtypes == 'int64':
#             df[i] = df[i].astype(np.int32)
# formatTrans(X_train,col)
# formatTrans(X_test,col)

# scalar=StandardScaler() 
# for i in col:
#     X_train[i]=scalar.fit_transform(X_train[i].values.reshape(-1, 1))
#     X_test[i]=scalar.transform(X_test[i].values.reshape(-1, 1))

# X_train.fillna(X_train.mean(),inplace=True)
# X_test.fillna(X_test.mean(),inplace=True)

In [ ]:
# SEED = 2019
# np.random.seed(SEED)
# random.seed(SEED)
# model = Sequential()
# model.add(Dense(2 ** 10,input_shape=[X_train.shape[1]], activation='relu'))
# model.add(Dropout(0.25))    
# model.add(BatchNormalization())
# model.add(Dense(2 ** 9, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.25)) 
# model.add(Dense(2 ** 5, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.25))      
# model.add(Dense(1,activation='sigmoid'))
# model.compile(loss='binary_crossentropy', optimizer='adam') 

# history = model.fit(X_train.values, y.values, epochs=10, batch_size=128, verbose=2, shuffle=True)
# model.summary()

In [ ]:
# lgb模型融合了五个种子会比较慢
col = [i for i in df_train.columns if i not in ['user','label']]
X_train = df_train[col]
#X_train['oof'] = oof
y = df_train['label']
X_test = df_test[col] 
#X_test['pre'] = prediction

oof = np.zeros(X_train.shape[0])
prediction = np.zeros(X_test.shape[0])
seeds = [4780, 7830, 4600, 5780, 6170]#(7388-7397)
num_model_seed = 5
for i,model_seed in enumerate(range(num_model_seed)):
    oof_cat = np.zeros(X_train.shape[0])
    prediction_cat=np.zeros(X_test.shape[0])
    skf = StratifiedKFold(n_splits=5, random_state=seeds[model_seed], shuffle=True)
    print(f'第{i}个种子：{model_seed}')
    for index, (train_index, test_index) in enumerate(skf.split(X_train, y)):
        print(index)
        train_x, test_x, train_y, test_y = X_train.iloc[train_index], X_train.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
        model = lgb.LGBMClassifier(objective='binary',
                           boosting_type='gbdt',
                           num_leaves=32,
                           max_depth=8,
                           learning_rate=0.01,
                           n_estimators=10000,
                           subsample=0.8,
                           feature_fraction=0.6,
                           reg_alpha=10,
                           reg_lambda=12,
                           random_state=2019,
                           is_unbalance=True,
                           metric='auc')
        model.fit(
            train_x, train_y,
            eval_set=[(train_x, train_y),(test_x, test_y)],
            eval_metric='auc',
            early_stopping_rounds=100,
            verbose=100)
        oof_cat[test_index] += model.predict_proba(test_x)[:,1]
        prediction_cat += model.predict_proba(X_test)[:, 1]/5
    print('AUC',roc_auc_score(y, oof_cat)) 
    oof += oof_cat / num_model_seed
    prediction += prediction_cat / num_model_seed
print('AUC',roc_auc_score(y, oof))

In [ ]:
su = df_test[['user']].copy()
su['prob'] = prediction
su.to_csv('submit.csv',index=False)

In [ ]:
submit['prob'] = submit['user'].map(dict(zip(df_test['user'], prediction)))
submit.to_csv('submit.csv',index=False)

In [ ]:
from itertools import combinations
combins = [c for c in  combinations([2008,2018,48,2000,450,70,90,150,420,30], 5)]

In [ ]:
import pandas as pd
import numpy as np
import gc
from sklearn.metrics import roc_auc_score
train_df = pd.read_csv('../input/train.csv', nrows=40000000)
test_df = pd.read_csv('../input/test.csv')
print("load done")
#dask_df = train_df
#df_pos = dask_df[(dask_df['is_attributed'] == 1)]
#df_neg = dask_df[(dask_df['is_attributed'] == 0)]
#df_pos = df_pos.sample(n=5000)
#print(len(df_pos))
#print(len(df_neg))
#df_neg = df_neg.sample(n=2000000)
#train_df = pd.concat([df_pos,df_neg]).sample(frac=1)
#del df_pos, df_neg
#gc.collect()
def remove_unkonwn_tag(col, train_df = train_df, test_df = test_df):
    test_df.loc[~test_df[col].isin(train_df[col]),col] = 9999999

def remove_lowfreq_tag(col, train_df = train_df,test_df = test_df, N=3):
    topN_address_list = train_df[col].value_counts()
    #print(topN_address_list)
    topN_address_list = topN_address_list[topN_address_list <= N]
    topN_address_list = topN_address_list.index
    remove_list = train_df.loc[train_df[col].isin(topN_address_list), col]
    print('remove:',len(remove_list))
    print('reserve',len(train_df) - len(remove_list))
    remove_list = 9999998
    test_df.loc[test_df[col].isin(topN_address_list), col] = 9999998

def preprocess(df):
    df['time_t'] = df.click_time.str[11:13] + df.click_time.str[14:16]

for i,j in zip(['app','ip','device','os','channel'],[5,4,4,5,5]):
    remove_lowfreq_tag(i,N=j)


for i in ['app','ip','device','os','channel']:
    remove_unkonwn_tag(i)

from sklearn.preprocessing import LabelEncoder
def process_lable_encoder(col, train_df = train_df, test_df = test_df):
    le = LabelEncoder()
    le.fit(np.hstack([train_df[col], test_df[col]]))
    train_df[col] = le.transform(train_df[col])
    test_df[col] = le.transform(test_df[col])

coding_list = ['ip','app','device','os','channel','time_t']

preprocess(train_df)
preprocess(test_df)
print("load done")

for i in coding_list:
    process_lable_encoder(i)
    
print("label gen done")
MAX_IP = np.max(train_df.ip.max()) + 2 #39612 #277396
MAX_DEVICE = np.max(train_df.device.max()) + 2 #299 #3475
MAX_OS = np.max(train_df.os.max()) + 2 #161 #3475
MAX_APP = np.max(train_df.app.max()) + 2 #214 #3475
MAX_CHANNEL = np.max(train_df.channel.max()) + 2  #155
MAX_TIME = np.max(train_df.time_t.max()) + 2 #24*60+1
print("MAX gen done")

def get_keras_data(df):
    X = {
        'ip': np.array(df.ip),
	    'app': np.array(df.app),
        'device': np.array(df.device),
        'os': np.array(df.os),
        'channel': np.array(df.channel),
        'clicktime': np.array(df.time_t),
    }
    return X

from keras.layers import Input, Dropout, Dense, concatenate, GRU, Embedding, Flatten, Activation
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K

def get_model(lr=0.001, decay=0.0):
    ip = Input(shape=[1], name="ip")
    app = Input(shape=[1], name="app")
    device = Input(shape=[1], name="device")
    os = Input(shape=[1], name="os")
    channel = Input(shape=[1], name="channel")
    clicktime = Input(shape=[1], name="clicktime")

    emb_ip = Embedding(MAX_IP, 64)(ip)
    emb_device = Embedding(MAX_DEVICE, 16)(device)
    emb_os= Embedding(MAX_OS, 16)(os)
    emb_app = Embedding(MAX_APP, 16)(app)
    emb_channel = Embedding(MAX_CHANNEL, 8)(channel)
    emb_time = Embedding(MAX_TIME, 32)(clicktime)

    main = concatenate([Flatten()(emb_ip), 
                        Flatten()(emb_device), 
                        Flatten()(emb_os),
                        Flatten()(emb_app),
                        Flatten()(emb_channel), 
                        Flatten()(emb_time)])
    main = Dense(128,kernel_initializer='normal', activation="tanh")(main)
    main = Dropout(0.2)(main)
    main = Dense(64,kernel_initializer='normal', activation="tanh")(main)
    main = Dropout(0.2)(main)    
    main = Dense(32,kernel_initializer='normal', activation="relu")(main)
    output = Dense(1,activation="sigmoid") (main)
    #model
    model = Model([ip, app, device, os, channel, clicktime], output)
    optimizer = Adam(lr=lr, decay=decay)
    model.compile(loss="binary_crossentropy", 
                  optimizer=optimizer)
    return model


from sklearn.model_selection import train_test_split


Y_train = train_df.is_attributed.values.reshape(-1, 1)

X_train, X_valid, y_train, y_valid = train_test_split(train_df[coding_list], Y_train, test_size = 0.1, random_state= 1984, stratify = Y_train)
X_train = get_keras_data(X_train[coding_list])
X_valid = get_keras_data(X_valid[coding_list])

print("Defining  model...")

# Model hyper parameters.
BATCH_SIZE = 1024*2
epochs = 1

# Calculate learning rate decay.
exp_decay = lambda init, fin, steps: (init/fin)**(1/(steps-1)) - 1
steps = int(len(train_df['ip']) / BATCH_SIZE) * epochs
lr_init, lr_fin = 0.001, 0.0003
lr_decay = exp_decay(lr_init, lr_fin, steps)

model = get_model(lr=lr_init, decay=lr_decay)

print("Fitting  model to training examples...")
cw = {0: 1, 1: 3}
for i in range(1):
    model.fit(
            X_train, y_train, epochs=1, batch_size=BATCH_SIZE,
            validation_data=(X_valid, y_valid), verbose=1,class_weight=cw
    )
    y_val_pred = model.predict(X_valid)[:, 0]
    print('Valid AUC: {:.4f}'.format(roc_auc_score(y_valid, y_val_pred)))

X_test = get_keras_data(test_df[coding_list])
preds = model.predict(X_test, batch_size=BATCH_SIZE)
sub = pd.DataFrame()
sub['click_id'] = test_df['click_id']
sub['is_attributed'] = preds
sub.to_csv('sub_nn.csv', index=False)
print(sub.head())

In [ ]:
2008 0.7380433590324023
2018 0.737785476264335
1 0.7374947713242492
48 0.7378598000922227
2000 0.7385279977371564
704

In [ ]:
420 AUC 0.7377168137895376
440 AUC 0.7376184863904467
450 AUC 0.7383320974716646
30 AUC 0.7377294852229016
40 AUC 0.7376122025478872
60 AUC 0.7373679673898517
70 AUC 0.7385521015197325
80 AUC 0.7373743986020773
90 AUC 0.7378648531033316
150 AUC 0.7381158036023088

In [ ]:
submit['prob'] = submit['user'].map(dict(zip(df_test['user'], prediction))) 
submit.to_csv('submit1.csv',index=False)

In [ ]:
1、AUC 0.7223035548204311, 0.684129
2、AUC 0.738926978118269  window27+23+15+gen_user_group_amount_features 70118
3、AUC 0.7392065631328348 window27+23+15+gen_user_group_amount_features+word2vec_daydiffs  70005
4、AUC 0.7395118234800954 window27+23+15+gen_user_group_amount_features+gen_user_countvec/tfidf_features(df=df_op, value='op_device') 70094
5、AUC 0.7400733349184808 gen_user_null_features+2 70054
6、AUC 0.7396464391857498 word2vec_op_device  70141
7、AUC 0.7396801408599353 word2vec_op_mo 降
8、AUC 0.7403186712226607 word2vec_op_type 降
9、AUC 0.7398117360769147 word2vec_op_channel 降
10、AUC 0.7402225579054323 word2vec_trans_platform 降
11、AUC 0.7409563055778169 word2vec_tunnel_in 降 70115
12、AUC 0.7389892812972378 window=9和2 702525
13、AUC 0.7396 gen_user_group_amount_features(df=df_trans, value='tunnel_in/tunnel_out' 70245
14、AUC 0.7399620955787308 gen_user_tfidf/countvec_features(df=df_trans, value='amount') 降
15、AUC 0.741150201936527 Word2vec32 降
16、AUC 0.740550201936527 gen_user_group_amount_features(df=df_trans, value='tunnel_in/tunnel_out')+Word2vec32 703591
17、融合1516 703702

In [ ]:
AUC 0.7406062286129351 
2次伪标签提示4个万分点